In [1]:
# prerequisites:
# pip install -U swifter[groupby]
import swifter

import pandas as pd
import numpy as np
import h5py 
import matplotlib.pyplot as plt
from scipy.stats import uniform
from sklearn import linear_model
import pickle

In [2]:
# HYPER-PARAMETERS 
NOISE_FACTOR = 2  # кол-во шумовых фотонов по отношению к правильным
MINIMUM_FOTONS_NUMBER = 4  # Не рассматриваем случаи с меньшим чем MINIMUM_FOTONS_NUMBER числом фотонов
NUMBER_OF_CASES = 1000 # Кол-во случаев для оценки
BAND_ROTATIONS_NUMBER = 100 # число поворотов для определения оптимального угла. минимальный угол поворота = 360гр/BAND_ROTATIONS_NUMBER 

In [3]:
df = pd.read_hdf("./data/Out_fin_pad(200)_x(-1.0,1.0)_E(0.01,10000.0)_eff(0.8)_102.h5")

In [4]:
svm = pickle.load(open("RBF SVM.model", "rb"))

In [5]:
# генерирует шум
def generate_uniform(df, size):
    size_df = len(df)
    xarr = uniform.rvs(scale = 199, size = size * size_df)
    yarr = uniform.rvs(scale = 199, size = size * size_df)
    return pd.DataFrame({'x_det_pad': xarr.astype(int), 'y_det_pad': yarr.astype(int)})

In [6]:
# определяется ли точка svm классификатором, после преобразований координат и поворота
def point_belongs_to_svm(px, py, x0, y0, alpha, svm):
    XY = [[px - x0, py - y0]]
    # матрица поворота
    R = np.transpose([
        [np.cos(alpha), np.sin(alpha)],
        [-np.sin(alpha),np.cos(alpha)]
    ])
    return svm.predict(np.matmul(XY,R))[0] == 1

In [7]:
# считает кол-во фотонов в центрированном датасете, найденных svm классификатором, после поворота
def count_points_in_svm(XY, alpha, svm):         
    # матрица поворота
    R = np.transpose([
        [np.cos(alpha), np.sin(alpha)],
        [-np.sin(alpha),np.cos(alpha)]
    ])        
    return svm.predict(np.matmul(XY,R)).sum()  

In [8]:
# находит оптимальный угол поворота для отдельного эксперимента путем полного перебора
def find_max_alfa_svm(df, x0, y0, svm):
    # смещаем координаты
    XY = df[['x_det_pad','y_det_pad']] - [x0,y0]
    
    alpha_max = 0
    cmax = 0
    for alpha in np.arange(-np.pi, np.pi, 2*np.pi/BAND_ROTATIONS_NUMBER):
        c = count_points_in_svm(XY, alpha, svm)
        if c > cmax:
            cmax = c
            alpha_max = alpha
            
    return alpha_max

In [9]:
# смещает и поворачивает данные отдельного
def rotate_data(df, x0, y0, alpha):          
    # смещаем и поворачиваем точки
    XY = df[['x_det_pad','y_det_pad']] - [x0,y0]
    # матрица поворота
    R = np.transpose([
        [np.cos(alpha), np.sin(alpha)],
        [-np.sin(alpha),np.cos(alpha)]
    ])      
    return np.matmul(XY,R)

In [10]:
# оцениваем точность метода для конкретного эксперимента
def evaluate_svm_method(df, svm):   
    # не рассматриваем случаи с малым числом фотонов
    if len(df) < MINIMUM_FOTONS_NUMBER:
        return
    
    # находим порождающую частицу
    particle = df.query('Eg == 0')
    # не рассматриваем случаи с двумя и более частицами
    if len(particle) > 1:
        return
    
    # находим координаты этой частицы
    x0 = float(particle.iloc[0]['x_det_pad'])
    y0 = float(particle.iloc[0]['y_det_pad'])
       
    # для оценки точности модели шум добавляем до определения угла поворота
    noise = generate_uniform(df, NOISE_FACTOR)
    df_and_noise = pd.concat([df, noise])
    
    # определяем нужный угол поворота
    alpha_max = find_max_alfa_svm(df_and_noise, x0, y0, svm)
    
    # крутим данные
    df_and_noise[['x_det_pad', 'y_det_pad']] = rotate_data(df_and_noise, x0, y0, alpha_max)
    
    # считаем метрики
    TP = 0 # True Positive (TP): you predicted positive, the real value was positive
    TN = 0 # True Negative (TN): you predicted negative, the real value was negative
    FP = 0 # False Positive (FP): you predicted positive, the real value was negative
    FN = 0 # False Positive (FP): you predicted positive, the real value was negative
    for index, row in df_and_noise.iterrows():
        if row['Eg'] == 0: # саму частицу не учитываем
            continue
        if svm.predict([[row['x_det_pad'],row['y_det_pad']]])[0] == 1:
            if np.isnan(row['Num']):
                FP += 1
            else:
                TP += 1
        else:
            if np.isnan(row['Num']):
                TN += 1
            else:
                FN += 1
                  
    return pd.DataFrame({'TP': [TP], 'TN': [TN], 'FP': [FP], 'FN': [FN]})

In [11]:
def plot_elements(df, low=0, up=200):
    fig, ax = plt.subplots(figsize=(5,5))
  
    plt.xlim(low,up)
    plt.ylim(low,up)
    particle = df.query('Eg == 0')
    df = df.drop(df[df.Eg == 0].index)
    
    data = df[df.Num.notnull()]
    noise = df[df.Num.isna()]
    
    ax.scatter(data[['x_det_pad']], data[['y_det_pad']], s=2)
    ax.scatter(noise[['x_det_pad']], noise[['y_det_pad']], s=1, facecolor='green')
    ax.scatter(particle[['x_det_pad']], particle[['y_det_pad']], facecolor='red', s=2)
    return fig, ax

In [12]:
# группируем данные по номеру эксперимента, для каждой группы запускаем оценку
evaluation = df.query('Num < @NUMBER_OF_CASES').swifter.groupby("Num").apply(lambda x: evaluate_svm_method(x, svm))

  0%|          | 0/24 [00:00<?, ?it/s]

2023-12-10 17:43:28,306	INFO worker.py:1673 -- Started a local Ray instance.


In [13]:
evaluation

,,TP,TN,FP,FN
Num,,,,,
0.0,0,5,9,3,0
1.0,0,9,19,1,0
2.0,0,7,11,7,1
3.0,0,1,6,2,2
4.0,0,2,6,2,1
...,...,...,...,...,...
994.0,0,6,13,1,0
995.0,0,1,5,3,2
996.0,0,4,9,1,0


In [14]:
total = evaluation.sum()

In [15]:
# Evaluation metrics: https://www.edlitera.com/blog/posts/evaluating-classification-models

# 1. Доля правильных предсказаний - Accuracy
# Accuracy = (Number of correct predictions) / (Overall number of predictions)

# 2. Точность - Precision
# Precision = (True Positive) / (True Positive + False Positive)

# 3. Полнота - Recall (Sensitivity, True Positive Rate)
# The recall is defined as the number of true positives divided by the sum of true positives and false negatives. 
# It expresses the ability to find all relevant instances in a dataset. 
# Recall measures how good your model is at correctly predicting positive cases. 
# It’s the proportion of actual positive cases which were correctly identified. The equation for recall is:
# Recall = (True Positive) / (True Positive + False Negative)

Accuracy = (total['TP'] + total['TN']) / (total['TP'] + total['TN'] + total['FP'] + total['FN'])
Precision = total['TP'] / (total['TP'] + total['FP'])
Recall = total['TP'] / (total['TP'] + total['FN'])
print('SVM METHOD SCORE:')
print('Accuracy:', Accuracy)
print('Precision:', Precision)
print('Recall:', Recall)

SVM METHOD SCORE:
Accuracy: 0.8493730170720653
Precision: 0.6931996855345912
Recall: 0.8906565656565657


In [16]:
import ray
ray.shutdown()